# Coroutine

async def f() - возвращает объект corщutine который можно запустить на исполнение (аналог Task на C#)

| Python	| C#	|
|---|---|
|async def f() |	async Task<int> f()|
|await f() |	await f()|
|Coroutine obj |	Task<int> (не завершён)|


In [1]:
import asyncio
async def f():
    print("Start")
    await asyncio.sleep(5)
    print("End")
    return 42

In [2]:
result = await f()
result

Start


End


42

# Async Generator и async for

Это асинхронный генератор.

Он создаёт значения постепенно, с возможностью await между значениями.

|Python Async Generator|	C#|	TypeScript (RxJS Observable)|
|---|---|---|
|async def ... yield |	IAsyncEnumerable<int>	|Observable<number>|
|async for x in gen |	await foreach(var x in gen)	|observable.subscribe(x => ...)|

In [3]:
import asyncio
from typing import Any, AsyncGenerator, Generator

## Generator
**!!! это НЕ асинхронный тип !!!**

Generator[YieldType, SendType, ReturnType]:
- YieldType — тип, который генерируется через yield.
- SendType — тип, который можно отправить внутрь генератора через метод .send().
- ReturnType — тип, который возвращает генератор при завершении (через return).

In [4]:
import time

def numbers(sleep_time: int = 1)->Generator[int,int,str]:
    for i in range(3):
        time.sleep(sleep_time)
        yield i
    return "end"

In [5]:
from datetime import datetime

for n in numbers(2):
    print(f"{n}: {datetime.now()}")


0: 2025-11-22 12:15:56.889403
1: 2025-11-22 12:15:58.889698
2: 2025-11-22 12:16:00.889882


## AsyncGenerator
AsyncGenerator[YieldType, SendType]:
- YieldType — тип, который генерируется через yield.
- SendType — тип, который можно отправить внутрь генератора через метод .send().

In [6]:
async def numbers(sleep_time: int = 1)->AsyncGenerator[int,int]:
    for i in range(3):
        await asyncio.sleep(sleep_time)
        yield i

In [7]:
from datetime import datetime

async for n in numbers(2):
    print(f"{n}: {datetime.now()}")


0: 2025-11-22 12:16:02.905456
1: 2025-11-22 12:16:04.907752
2: 2025-11-22 12:16:06.910017


# Объекты с __aiter__

Любой объект в Python, у которого есть метод __aiter__, можно использовать в async for.

async for — это способ «подписки» на поток значений, как Observable.subscribe или await foreach в C

Здесь AsyncCounter полностью аналог Observable или IAsyncEnumerable — поток значений, которые приходят асинхронно.

In [8]:
class AsyncCounter:
    def __init__(self, n):
        self.n = n
    def __aiter__(self):
        self.i = 0
        return self
    async def __anext__(self):
        if self.i >= self.n:
            raise StopAsyncIteration
        await asyncio.sleep(2)
        self.i += 1
        return self.i - 1

In [9]:
async for x in AsyncCounter(3):
    print(f"{x}: {datetime.now()}")

0: 2025-11-22 12:16:08.925184
1: 2025-11-22 12:16:10.927438
2: 2025-11-22 12:16:12.929699


In [10]:
from typing import AsyncIterable


def get_counter() -> AsyncIterable[int]:
    return AsyncCounter(3)

In [11]:
async for x in get_counter():
    print(f"{x}: {datetime.now()}")

0: 2025-11-22 12:16:14.945229
1: 2025-11-22 12:16:16.947422
2: 2025-11-22 12:16:18.949569
